<a href="https://colab.research.google.com/github/Esmreety/Disease-Prediction/blob/master/Predicting_lung_disease_(transfer_learning_VGG_16).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import  layers, Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import backend as k
from keras.preprocessing import image




In [9]:

train_path = '/content/chest_xray/train'
valid_path = '/content/chest_xray/val'
vgg = VGG16(input_shape =(224, 224, 3),
                          include_top = False,
                          weights = 'imagenet')
for layer in vgg.layers:
  layer.trainable = False 


In [10]:
x = Flatten()(vgg.output)
prediction = Dense(2,activation = 'softmax')(x)
#create a model object
model = Model(inputs = vgg.input, outputs = prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [11]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [12]:
train_datagen = ImageDataGenerator(
                     rescale = 1.0/255.,
                     rotation_range = 40, 
                     width_shift_range = 0.2,
                     height_shift_range = 0.2,
                     shear_range = 0.2,
                     zoom_range = 0.2,
                     horizontal_flip = True,
                     fill_mode = 'nearest')
                    
              
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [15]:
train_generator = train_datagen.flow_from_directory(
                              train_path,
                              target_size = (224,224),
                              batch_size = 32,
                                class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(
                                valid_path,
                                target_size = (224,224),
                                batch_size = 32,
                                class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [16]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = len(train_generator),
    validation_data = validation_generator,
    epochs = 5,
    validation_steps = len(validation_generator)) 

Epoch 1/5
163/163 [==============================] - 130s 797ms/step - loss: 0.2552 - accuracy: 0.8919 - val_loss: 0.2745 - val_accuracy: 0.8125
Epoch 2/5
163/163 [==============================] - 122s 749ms/step - loss: 0.1823 - accuracy: 0.9260 - val_loss: 0.3537 - val_accuracy: 0.7500
Epoch 3/5
163/163 [==============================] - 123s 756ms/step - loss: 0.1551 - accuracy: 0.9410 - val_loss: 0.3226 - val_accuracy: 0.8125
Epoch 4/5
163/163 [==============================] - 124s 758ms/step - loss: 0.1364 - accuracy: 0.9473 - val_loss: 0.2430 - val_accuracy: 0.8750
Epoch 5/5
163/163 [==============================] - 124s 758ms/step - loss: 0.1386 - accuracy: 0.9440 - val_loss: 0.5812 - val_accuracy: 0.6875


In [21]:
model.save('model_predict.hdf5')


In [25]:
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
model = load_model('/content/model_predict.h5')
img = image.load_img('/content/chest_xray/test/PNEUMONIA/person101_bacteria_483.jpeg', target_size = (224, 224))
x = image.img_to_array(img)
x  = np.expand_dims(x, axis = 0, )
img_data = preprocess_input(x)
classes = model.predict(img_data)
print(classes)

[[0. 1.]]
